In [1]:
import os
import time
import joblib
import shutil
import tarfile
import requests

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.signal import butter, lfilter

from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.utils import to_categorical

from sklearn.utils import shuffle
from sklearn.metrics import classification_report, accuracy_score

sns.set(font_scale=2)
sns.set_style('white')

In [2]:

DATASET_ID      = '1p0CSRb9gax0sKqdyzOYVt-BXvZ4GtrBv'

# -------------BASE DIR (MODIFY THIS TO YOUR NEED) ------------ #
# BASE_DIR        = '../'
BASE_DIR        = '/content'

DATA_DIR        = 'Sensor-Data/'
CHANNELS_DIR    = 'Channels/'
FEATURES_DIR    = 'Features/'
FIGURE_DIR      = 'Figures/'
LOG_DIR         = 'Logs/'

USERS           = ['001', '002', '003', '004', '005', '006', '007', '008', '009',
                   '010', '011', '012', '013', '014', '015', '016', '017', '018',
                   '019', '020', '021', '022', '023', '024', '025']
GESTURES        = ['j', 'z', 'bad', 'deaf', 'fine', 'good', 'goodbye', 'hello', 'hungry',
                   'me', 'no', 'please', 'sorry', 'thankyou', 'yes', 'you']

WINDOW_LEN      = 150

# ------------- FOR THE GREATER GOOD :) ------------- #
DATASET_LEN     = 1120
TRAIN_LEN       = 960
TEST_LEN        = 160

TEST_USER       = '001'
EPOCHS          = 5

CHANNELS_GROUP  = 'DYNAMIC_ACC_ONLY_'
CUT_OFF         = 3.0
ORDER           = 4
FS              = 100

CONFIG          = "Rolling median filter for flex, LPF for IMU, Stacked CNN, epochs 20, lr 0.001\n"

In [3]:

#--------------------- Download util for Google Drive ------------------- #

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)
        
    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value
        
    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk:
                f.write(chunk)

def download_data(fid, destination):
    print('cleaning already existing files ... ', end='')
    try:
        shutil.rmtree(destination)
        print('√')
    except:
        print('✕')
        
    print('creating data directory ... ', end='')
    os.mkdir(destination)
    print('√')
    
    print('downloading dataset from the repository ... ', end='')
    filename = os.path.join(destination, 'dataset.tar.xz')
    try:
        download_file_from_google_drive(fid, filename)
        print('√')
    except:
        print('✕')
        
    print('extracting the dataset ... ', end='')
    try:
        tar = tarfile.open(filename)
        tar.extractall(destination)
        tar.close()
        print('√')
    except:
        print('✕')

In [4]:

# ------- Comment This if already downloaded -------- #

destination = os.path.join(BASE_DIR, DATA_DIR)
download_data(DATASET_ID, destination)

cleaning already existing files ... ✕
creating data directory ... √
downloading dataset from the repository ... √
extracting the dataset ... √


In [7]:
class LowPassFilter(object): 
    def butter_lowpass(cutoff, fs, order):
        nyq = 0.5 * fs
        normal_cutoff = cutoff / nyq
        b, a = butter(order, normal_cutoff, btype='low', analog=False)
        return b, a

    def apply(data, cutoff=CUT_OFF, fs=FS, order=ORDER):
        b, a = LowPassFilter.butter_lowpass(cutoff, fs, order=order)
        y = lfilter(b, a, data)
        return y

In [5]:
def clean_dir(path):
    print('cleaning already existing files ... ', end='')
    try:
        shutil.rmtree(path)
        print('√')
    except:
        print('✕')
    
    print('creating ' + path + ' directory ... ', end='')
    os.mkdir(path)
    print('√')

def extract_channels():
    channels_dir = os.path.join(BASE_DIR, CHANNELS_DIR)
    clean_dir(channels_dir)
        
    for user in USERS:
        print('Processing data for user ' + user, end=' ')
        
        X = []
        y = []
        first_time = True
        
        for gesture in GESTURES:
              
            user_dir = os.path.join(BASE_DIR, DATA_DIR, user)
            gesture_dir = os.path.join(user_dir, gesture + '.csv')

            dataset = pd.read_csv(gesture_dir)

            dataset['flex_1'] = dataset['flex_1'].rolling(3).median()
            dataset['flex_2'] = dataset['flex_2'].rolling(3).median()
            dataset['flex_3'] = dataset['flex_3'].rolling(3).median()
            dataset['flex_4'] = dataset['flex_4'].rolling(3).median()
            dataset['flex_5'] = dataset['flex_5'].rolling(3).median()

            dataset.fillna(0, inplace=True)

            # flex = ['flex_1', 'flex_2', 'flex_3', 'flex_4', 'flex_5']
            # max_flex = dataset[flex].max(axis=1)
            # max_flex.replace(0, 1, inplace=True)
            # dataset[flex] = dataset[flex].divide(max_flex, axis=0)
            
            flx1 = dataset['flex_1'].to_numpy().reshape(-1, WINDOW_LEN)
            flx2 = dataset['flex_2'].to_numpy().reshape(-1, WINDOW_LEN)
            flx3 = dataset['flex_3'].to_numpy().reshape(-1, WINDOW_LEN)
            flx4 = dataset['flex_4'].to_numpy().reshape(-1, WINDOW_LEN)
            flx5 = dataset['flex_5'].to_numpy().reshape(-1, WINDOW_LEN)
            
            accx = dataset['ACCx'].to_numpy()
            accy = dataset['ACCy'].to_numpy()
            accz = dataset['ACCz'].to_numpy()
            
            accx = LowPassFilter.apply(accx).reshape(-1, WINDOW_LEN)
            accy = LowPassFilter.apply(accy).reshape(-1, WINDOW_LEN)
            accz = LowPassFilter.apply(accz).reshape(-1, WINDOW_LEN)
            
            gyrx = dataset['GYRx'].to_numpy()
            gyry = dataset['GYRy'].to_numpy()
            gyrz = dataset['GYRz'].to_numpy()
            
            gyrx = LowPassFilter.apply(gyrx).reshape(-1, WINDOW_LEN)
            gyry = LowPassFilter.apply(gyry).reshape(-1, WINDOW_LEN)
            gyrz = LowPassFilter.apply(gyrz).reshape(-1, WINDOW_LEN)
            
            accm = np.sqrt(accx ** 2 + accy ** 2 + accz ** 2)
            gyrm = np.sqrt(gyrx ** 2 + gyry ** 2 + gyrz ** 2)
            
            g_idx = GESTURES.index(gesture)
            labels = np.ones((accx.shape[0], 1)) * g_idx
            
            channels = np.stack([
                flx1, flx2, flx3, flx4, flx5,
                accx, accy, accz
            ], axis=-1)
            
            if first_time == True:
                X = channels
                y = labels
                first_time = False
            else:
                X = np.append(X, channels, axis=0)
                y = np.append(y, labels, axis=0)
            
        
        x_path = os.path.join(BASE_DIR, CHANNELS_DIR, CHANNELS_GROUP + user + '_X.joblib')
        y_path = os.path.join(BASE_DIR, CHANNELS_DIR, CHANNELS_GROUP + user + '_y.joblib')
        joblib.dump(X, x_path)
        joblib.dump(y, y_path)
        
        print('√')
        

In [8]:
extract_channels()

cleaning already existing files ... √
creating /content/Channels/ directory ... √
Processing data for user 001 √
Processing data for user 002 √
Processing data for user 003 √
Processing data for user 004 √
Processing data for user 005 √
Processing data for user 006 √
Processing data for user 007 √
Processing data for user 008 √
Processing data for user 009 √
Processing data for user 010 √
Processing data for user 011 √
Processing data for user 012 √
Processing data for user 013 √
Processing data for user 014 √
Processing data for user 015 √
Processing data for user 016 √
Processing data for user 017 √
Processing data for user 018 √
Processing data for user 019 √
Processing data for user 020 √
Processing data for user 021 √
Processing data for user 022 √
Processing data for user 023 √
Processing data for user 024 √
Processing data for user 025 √


In [9]:
X

NameError: ignored